Creating lower bounds by finding composite boards that give valid results

In [1]:
from itertools import product
from tqdm.notebook import tqdm

In [2]:
class NQueen:
    def __init__(self, N):
        self.N = N
        self.solutions = []
        
    def solve(self):
        board = [[0] * self.N for _ in range(self.N)]
        self._solve(board)
    
    def _solve(self, board, row=0, cols=set(), diags=set(), anti_diags=set()):
        if row == self.N:
            return self.solutions.append([row.copy() for row in board])
            
        for col in range(self.N):
            if col in cols or (row-col) in diags or (row+col) in anti_diags: continue
            
            cols.add(col)
            diags.add(row-col)
            anti_diags.add(row+col)
            board[row][col] = 1
            
            self._solve(board, row+1, cols, diags, anti_diags)
            
            board[row][col] = 0
            cols.remove(col)
            diags.remove(row-col)
            anti_diags.remove(row+col)
            
    @staticmethod
    def is_valid(board):
        N = len(board)
        diags = set()
        anti_diags = set()
        rows = set()
        cols = set()
        
        for row in range(N):
            for col in range(N):
                if board[row][col]:
                    if row in rows or col in cols or col-row in diags or row+col in anti_diags: return False
                    rows.add(row)
                    cols.add(col)
                    diags.add(col-row)
                    anti_diags.add(row+col)
                    
        return len(rows) == N

In [3]:
solver = NQueen(4)
solver.solve()
len(solver.solutions)

2

In [4]:
solver = NQueen(5)
solver.solve()
len(solver.solutions)

10

In [5]:
NQueen.is_valid(solver.solutions[0])

True

# Composition finder

In [6]:
inner_size = 5
outer_size = 5

solver_inner = NQueen(inner_size)
solver_inner.solve()
sols_inner = solver_inner.solutions
print(len(sols_inner))

solver_outer = NQueen(outer_size)
solver_outer.solve()
sols_outer = solver_outer.solutions
print(len(sols_outer))

10
10


In [7]:
sols_outer[0]

[[1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0],
 [0, 0, 0, 1, 0]]

In [8]:
solutions = []
zero_mat = [[0] * inner_size for _ in range(inner_size)]
for _sol_outer in tqdm(sols_outer, desc='Progress of outer loop'):
    sol_inner_combis = list(product(*[sols_inner for _ in range(outer_size)]))

    inner_pbar = tqdm(sol_inner_combis, desc='Progress of inner loop', leave=False)
    for combi in inner_pbar:
        sol_outer = [row.copy() for row in _sol_outer]
        pointer = 0
        for row in range(outer_size):
            for col in range(outer_size):
                if not sol_outer[row][col]:
                    sol_outer[row][col] = zero_mat
                else:
                    sol_outer[row][col] = combi[pointer]
                    pointer += 1
            
        # Turn into 2D (outer_size x inner_size)^2 matrix
        new_size = inner_size * outer_size
        new_mat = [[0] * new_size for _ in range(new_size)]
        for outer_row in range(outer_size):
            for outer_col in range(outer_size):
                for inner_row in range(inner_size):
                    for inner_col in range(inner_size):
                        new_mat[outer_row * inner_size + inner_row][outer_col * inner_size + inner_col] = sol_outer[outer_row][outer_col][inner_row][inner_col]
                        
        if NQueen.is_valid(new_mat):
            solutions.append(new_mat)
            inner_pbar.set_description(f"Progress of inner loop (Solutions: {len(solutions)})")
    
    tqdm.write(f"Progress after this outer iteration: {len(solutions)} solutions found.")

print(f"Total solutions found: {len(solutions)}")

Progress of outer loop:   0%|          | 0/10 [00:00<?, ?it/s]

Progress of inner loop:   0%|          | 0/100000 [00:00<?, ?it/s]

Progress after this outer iteration: 352 solutions found.


Progress of inner loop:   0%|          | 0/100000 [00:00<?, ?it/s]

Progress after this outer iteration: 704 solutions found.


...

Progress after this outer iteration: 3200 solutions found.


Progress of inner loop:   0%|          | 0/100000 [00:00<?, ?it/s]

Progress after this outer iteration: 3552 solutions found.


Progress of inner loop:   0%|          | 0/100000 [00:00<?, ?it/s]

Progress after this outer iteration: 3904 solutions found.
Total solutions found: 3904


In [9]:
NQueen.is_valid(solutions[0])

True

In [10]:
solutions[0]

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,